In [1]:
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
import os
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
from subprocess import run, PIPE
import sys
 
import dask # Distributed data libary
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
import xarray as xr # Libary to work with labeled n-dimensional data and dask
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# Set some user specific variables
scratch_dir = Path('/scratch') / getuser()[0] / getuser() # Define the users scratch dir

# Create a temp directory where the output of distributed cluster will be written to, after this notebook
# is closed the temp directory will be closed
dask_tmp_dir = TemporaryDirectory(dir=scratch_dir, prefix='threshold_')
cluster = SLURMCluster(memory='500GiB',
                       cores=72,
                       project='mh0731',
                       walltime='01:45:00',
                       queue='gpu',
                       name='threshold',
                       scheduler_options={'dashboard_address': ':12435'},
                       local_directory=dask_tmp_dir.name,
                       job_extra=[f'-J thrshld', 
                                  f'-D {dask_tmp_dir.name}',
                                  f'--begin=now',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o'
                                 ],
                       interface='ib0')

cluster.scale(jobs=2) # requests whole nodes
dask_client = Client(cluster)
dask_client.wait_for_workers(18) # gpu-partition has 9 workers per node

In [4]:
data_path = Path('/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/')
glob_pattern_2d = 'pr_*[0-9]_reggrid.nc'
 
# Collect all file names with pathlib's rglob and list compressions 
# dont take first ten days, they are spin-up
file_names = sorted([str(f) for f in data_path.rglob(f'{glob_pattern_2d}')])[10:-1]

In [5]:
file_names

['/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200131T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200201T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200202T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200203T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200204T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200205T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200206T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200207T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200208T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200209T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200210T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/pr_20200211T0000_reggrid.nc',
 '/work/mh0731/m300414/DyWin

Radar data is in mm/hour and model data in kg/m2s. Conversion factor to get from mm/hour to kg/m2s is 1/3600.

In [6]:
# opening all consecutive data does not work well later with the numpy-arrays
# ds = xr.open_mfdataset(file_names, combine='by_coords', parallel=True) # doesnt want to work, maybe too much although in dask
# ds

# rain_threshold_in_mm = 5
# rain_threshold       = rain_threshold_in_mm / 3600

@dask.delayed
def get_convective_rain(file, rain_threshold):
    rain          = xr.open_dataset(file)['pr']
    assert rain.dims[1] in ('lat', 'lon')
    assert rain.dims[2] in ('lat', 'lon')
    coast_mask = xr.open_dataarray(
        '/work/mh0731/m300414/DyWinter_b10/Cartesian_Grid/northwestaustralia_mask.nc').squeeze()

    rain          = rain.where(coast_mask)
    conv_mask     = rain > rain_threshold
    
    # everython per time step
    n_conv_cells   = conv_mask.sum(dim=('lat', 'lon'))
    n_cells_domain = coast_mask.sum(dim=('lat', 'lon'))
    conv_area_fraction = n_conv_cells / n_cells_domain
    avg_conv_rain      = xr.where(conv_mask, rain, 0.).sum(dim=('lat', 'lon')) / n_cells_domain
    
    # # everything for the whole file (time and space)
    # n_rainy_cells = (rain > 0.).sum() 
    # n_conv_cells  = conv_mask.sum()
    # conv_rain_pixels    = rain[conv_mask] # boolean indexing of 2d-numpy-array returns just 1d-vector
    # conv_to_strat_ratio = n_conv_cells / n_rainy_cells
    
    assert conv_area_fraction.max() < 1.
    
    return (avg_conv_rain , conv_area_fraction)
    # return (conv_rain_pixels , conv_to_strat_ratio)

Bring data into distributed memory via persist()

In [7]:
def run_data(rain_threshold_in_mm):
    rain_threshold = rain_threshold_in_mm / 3600
    
    rain_futures = []
    for file in file_names:
        rain_futures.append(get_convective_rain(file, rain_threshold))
    
    # Bring data into distributed memory via persist()
    jobs = dask.persist(rain_futures)
    progress(jobs, notebook=False)
    
    return rain_futures

Gather data into single memory with compute(). Here this acts on data which already was brought into distributed memory via persist(), thus it should be fast.

In [8]:
def gather_data(futures):
    rain_tuples = dask.compute(*futures)

    mean_conv_rain, caf = zip(*rain_tuples)

    mean_conv_rain     = xr.concat(mean_conv_rain, dim='time') * 3600
    conv_area_fraction = xr.concat(caf, dim='time')
    
    return mean_conv_rain, conv_area_fraction

In [9]:
def plot_data(mean_conv_rain, conv_area_fraction, threshold):
    r = np.corrcoef(mean_conv_rain, conv_area_fraction)[0, 1]
    plt.hist2d(conv_area_fraction*100, mean_conv_rain, bins=(50, 50))
    plt.colorbar()
    plt.xlabel('Convective area fraction [%]')
    plt.ylabel("Domain-average 'convective' rain rate [mm/h]")
    plt.title(f"'Convective' rain > {threshold:.2f} mm/h, r={r:.2f}")
    plt.savefig(f'/home/mpim/m300414/Plots/australia_caf_convrain_higher{threshold:.2f}.pdf', bbox_inches='tight', transparent=True)
    plt.close()

In [10]:
# pr = xr.open_mfdataset(file_names)
# pr

In [11]:
thresholds = np.arange(14, 19, 1)
for threshold in [15]: #thresholds:
    dask_futures = run_data(threshold)
    conv_rr, caf = gather_data(dask_futures)
    plot_data(conv_rr, caf, threshold)

    # tuples = dask.compute(*dask_futures)
    # conv_rain, conv_percentage = zip(*tuples)

distributed.scheduler - ERROR - Couldn't gather keys {'get_convective_rain-e952371c-40ad-455b-8aad-4cba107ec84c': ['tcp://10.50.34.200:33569']} state: ['memory'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://10.50.34.200:33569'], get_convective_rain-e952371c-40ad-455b-8aad-4cba107ec84c
NoneType: None


In [12]:
# conv_rain = np.concatenate(conv_rain)

# print(f'{threshold/3600} should be smaller than {conv_rain.min()}')
# print(f'On average {np.asarray(conv_percentage).mean()*100}% of rainy cells are of convective nature, given a rain threshold of {threshold} mm/hour.')

# ds = xr.open_dataset(file_names[0])
# ds_out = xr.DataArray(conv_rain, name=ds['pr'].name, attrs={**ds['pr'].attrs,**ds.attrs})
# ds_out.to_netcdf('/work/mh0731/m300414/DyWinter_b10/Convective_Values/conv_rain_values_7mmhour.nc')

In [13]:
conv_rr.name = 'avg_conv_rain'
conv_rr.attrs['units'] = 'mm/hour'
conv_rr.to_netcdf('/work/mh0731/m300414/DyWinter_b10/Convective_Values/australia_avg_conv_rain_15mmhour.nc')

In [14]:
caf_p = caf*100
caf_p.name = 'conv_area_fraction'
caf_p.attrs['units'] = '%'
caf_p.to_netcdf('/work/mh0731/m300414/DyWinter_b10/Convective_Values/australia_conv_area_fraction_15mmhour.nc')

In [15]:
caf.max()

<xarray.DataArray ()>
array(0.02758659)
Coordinates:
    plev     float64 9e+04

In [16]:
conv_rr.max()

<xarray.DataArray 'avg_conv_rain' ()>
array(0.96165896)
Coordinates:
    plev     float64 9e+04